# Twitter  Bot

This will:

- Send predictions at game start for each game being played
- Send a summary of the day's games at the end of the day

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import random
import shutil
import plotly
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests     
from scipy.stats import norm
import winsound
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import WebDriverException
import tweepy
import re

In [2]:
api_key = os.environ.get('Twitter_API_Key')
api_secret = os.environ.get('Twitter_API_Secret_Key')
access_token = os.environ.get('Twitter_Access_Token')
access_token_secret = os.environ.get('Twitter_Access_token_Secret')
bearer_token = os.environ.get('Twitter_Bearer_Token')

client = tweepy.Client(bearer_token, api_key, api_secret, access_token, access_token_secret)

auth = tweepy.OAuthHandler(api_key, api_secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
# api.update_status('This is a test. Hey, yall.')

Status(_api=<tweepy.api.API object at 0x000002C9517D6620>, _json={'created_at': 'Fri Jan 13 20:03:55 +0000 2023', 'id': 1613990245326008322, 'id_str': '1613990245326008322', 'text': 'This is a test. Hey, yall.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://www.twitter.com" rel="nofollow">ML_AutoPredict</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1613983720314408960, 'id_str': '1613983720314408960', 'name': 'Machine_Learning_Sports_Predictions', 'screen_name': 'SportsML_', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 1, 'listed_count': 0, 'created_at': 'Fri Jan 13 19:38:07 +0000 2023', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, '

In [4]:
team_name_dict= [{'team': 'Atlanta', 'abbreviation': 'ATL'},
                 {'team': 'Boston', 'abbreviation': 'BOS'},
                 {'team': 'Brooklyn', 'abbreviation': 'BKN'},
                 {'team': 'Charlotte', 'abbreviation': 'CHA'},
                 {'team': 'Chicago', 'abbreviation': 'CHI'},
                 {'team': 'Cleveland', 'abbreviation': 'CLE'},
                 {'team': 'Dallas', 'abbreviation': 'DAL'},
                 {'team': 'Denver', 'abbreviation': 'DEN'},
                 {'team': 'Detroit', 'abbreviation': 'DET'},
                 {'team': 'Golden State', 'abbreviation': 'GSW'},
                 {'team': 'Houston', 'abbreviation': 'HOU'},
                 {'team': 'Indiana', 'abbreviation': 'IND'},
                 {'team': 'LA Clippers', 'abbreviation': 'LAC'},
                 {'team': 'LA Lakers', 'abbreviation': 'LAL'},
                 {'team': 'Memphis', 'abbreviation': 'MEM'},
                 {'team': 'Miami', 'abbreviation': 'MIA'},
                 {'team': 'Milwaukee', 'abbreviation': 'MIL'},
                 {'team': 'Minnesota', 'abbreviation': 'MIN'},
                 {'team': 'New Orleans', 'abbreviation': 'NOP'},
                 {'team': 'New York', 'abbreviation': 'NYK'},
                 {'team': 'Oklahoma City', 'abbreviation': 'OKC'},
                 {'team': 'Orlando', 'abbreviation': 'ORL'},
                 {'team': 'Philadelphia', 'abbreviation': 'PHI'},
                 {'team': 'Phoenix', 'abbreviation': 'PHX'},
                 {'team': 'Portland', 'abbreviation': 'POR'},
                 {'team': 'Sacramento', 'abbreviation': 'SAC'},
                 {'team': 'San Antonio', 'abbreviation': 'SAS'},
                 {'team': 'Toronto', 'abbreviation': 'TOR'},
                 {'team': 'Utah', 'abbreviation': 'UTA'},
                 {'team': 'Washington', 'abbreviation': 'WAS'}]

#### Check if game in next 30 minutes

In [5]:
# get current time
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")
# make current time normal, not military time
current_time = datetime.datetime.strptime(current_time, '%H:%M:%S').strftime('%I:%M %p')
#
print(current_time)

12:04 PM


In [6]:
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
odds_files = os.listdir('data/team/aggregates/daily_updates/')
odds_files = [file for file in odds_files if 'games_and_odds' in file]

if today in odds_files[0]:
    print('found today')
    big_odds_df = pd.read_csv('data/team/aggregates/daily_updates/' + odds_files[0])
    
else:

    # use oddsshark scrape
    url = 'https://www.oddsshark.com/nba/odds'

    driver = webdriver.Chrome()
    driver.get(url)

    # Get the html of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    data = soup.find('div', class_= 'odds--group__events-container')
    single_games = data.find_all(class_ = 'odds--group__event-container basketball')
    big_odds_df = pd.DataFrame()
    for game in single_games:
        # get teams
        teams = game.find(class_ = 'odds--group__event-participants').text.strip()
        teams = teams.replace('vs', '')
        teams = teams.replace(' ET', '')
        teams = teams.replace('  ', ' ')
        # get team 1
        # find index of first number
        index = 0
        for char in teams:
            if char.isdigit():
                break
            index += 1
        team1 = teams[:index].strip()
        # get time... from index of first number through "pm"
        pm = teams.find('pm')
        time = teams[index:pm]
        # add 12 hours to time if pm
        # get team 2
        team2 = teams[pm+2:].strip()
        # get odds
        spreads = game.find_all('div', 'odds-spread')

        odds_df = pd.DataFrame()

        n = 1
        for spread in spreads:
            spread = spread.text.replace('- ', '').replace(' - ', '').strip()[:-2][:-4]
            # opening spread
            if n == 1:
                opening_spread_t1 = spread
                n += 1
            elif n == 2:
                opening_spread_t2 = spread
                n += 1
            elif n == 3:
                current_spread_t1 = spread
                n += 1
            elif n == 4:
                current_spread_t2 = spread
                n += 1
        
        moneylines = game.find_all('div', 'odds-moneyline above-tablet-only')
        n = 1
        for moneyline in moneylines:
            moneyline = moneyline.text.replace('- ', '').replace(' - ', '').strip()
            # opening moneyline
            if n == 1:
                opening_moneyline_t1 = moneyline
                n += 1
            elif n == 2:
                opening_moneyline_t2 = moneyline
                n += 1
            elif n == 3:
                current_moneyline_t1 = moneyline
                n += 1
            elif n == 4:
                current_moneyline_t2 = moneyline
                n += 1

                # add to dataframe
                odds_df = odds_df.append({'team1': team1, 'team2': team2, 'time': time, 'opening_spread_t1': opening_spread_t1, 'opening_spread_t2': opening_spread_t2, 'current_spread_t1': current_spread_t1, 'current_spread_t2': current_spread_t2, 'opening_moneyline_t1': opening_moneyline_t1, 'opening_moneyline_t2': opening_moneyline_t2, 'current_moneyline_t1': current_moneyline_t1, 'current_moneyline_t2': current_moneyline_t2}, ignore_index=True)

        big_odds_df = big_odds_df.append(odds_df, ignore_index=True)

    # close driver
    driver.close()

    today = datetime.date.today()
    today = today.strftime('%Y-%m-%d')
    time_now = datetime.datetime.now().strftime('%H-%M')
    big_odds_df.to_csv('data/team/aggregates/daily_updates/games_and_odds_' + str(today) +'_'+ str(time_now) + '.csv')

big_odds_df


,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2
0,New York,Washington,7:00,-4,+4,-4,+4,-150,+125,-173,+153
1,Atlanta,Indiana,7:00,-3,+3,-2,+2,-150,+125,-126,+106
2,New Orleans,Detroit,7:00,-5,+5,-5.5,+5.5,-233,+165,-212,+182
3,Golden State,San Antonio,7:30,-8.5,+8.5,-8.5,+8.5,-400,+260,-365,+295
4,Oklahoma City,Chicago,8:00,+6,-6,+4.5,-4.5,+180,-263,+152,-172
5,Phoenix,Minnesota,8:00,+2.5,-2.5,+5,-5,+120,-147,+170,-195
6,Orlando,Utah,9:00,+5,-5,+6,-6,+165,-233,+195,-225
7,Denver,LA Clippers,10:00,-0.5,+0.5,+4.5,-4.5,-111,-110,+162,-182


In [7]:
# add 12 hours to time
big_odds_df['time'] = big_odds_df['time'] + ' PM'
# add 12 hours
big_odds_df['time'] = pd.to_datetime(big_odds_df['time'], format='%I:%M %p').dt.time
big_odds_df

,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2
0,New York,Washington,19:00:00,-4,+4,-4,+4,-150,+125,-173,+153
1,Atlanta,Indiana,19:00:00,-3,+3,-2,+2,-150,+125,-126,+106
2,New Orleans,Detroit,19:00:00,-5,+5,-5.5,+5.5,-233,+165,-212,+182
3,Golden State,San Antonio,19:30:00,-8.5,+8.5,-8.5,+8.5,-400,+260,-365,+295
4,Oklahoma City,Chicago,20:00:00,+6,-6,+4.5,-4.5,+180,-263,+152,-172
5,Phoenix,Minnesota,20:00:00,+2.5,-2.5,+5,-5,+120,-147,+170,-195
6,Orlando,Utah,21:00:00,+5,-5,+6,-6,+165,-233,+195,-225
7,Denver,LA Clippers,22:00:00,-0.5,+0.5,+4.5,-4.5,-111,-110,+162,-182


In [8]:
now = datetime.datetime.now()
now

datetime.datetime(2023, 1, 13, 12, 4, 42, 860738)

In [10]:
# add 30 minutes to current time
now1 = pd.to_datetime(now)
in_30 = now + datetime.timedelta(minutes=30)


In [14]:
now1 = now1.time()
now1

datetime.time(12, 4, 42, 860738)

In [11]:
in_30 = pd.to_datetime(in_30).time()
in_30

datetime.time(12, 34, 42, 860738)

In [16]:

games_soon = big_odds_df[big_odds_df['time'] > now1 ]
games_soon = games_soon[games_soon['time'] < in_30]
games_soon

,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2


#### Check if games have confirmed lineups

In [19]:
if len(games_soon) > 0:

    driver = webdriver.Chrome()
    url = 'https://www.rotowire.com/basketball/nba-lineups.php'


    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # find div class = 'lineups'. This is the div that contains all the lineups.
    # The lineups are split by 'lineup is-nba' class

    tot_lineup_df = pd.DataFrame()

    lineups = soup.find_all('div', class_='lineup__box')
    # get rid of last 4 lineups
    lineups = lineups[:-4]

    for lineup in lineups:
        # wait for page to load

        away_team = lineup.find('ul', class_= 'lineup__list is-visit').text.strip().replace('\n', ' ')
        home_team = lineup.find('ul', class_= 'lineup__list is-home').text.strip().replace('\n', ' ')

            # split into three lists by 'Expected Lineup'
        away_team_name = lineup.find('a', class_='lineup__team is-visit').text.strip()
        home_team_name = lineup.find('a', class_='lineup__team is-home').text.strip()

        away_team_pg = away_team[away_team.find('PG')+3:away_team.find('SG')-1]
        away_team_sg = away_team[away_team.find('SG')+3:away_team.find('SF')-1]
        away_team_sf = away_team[away_team.find('SF')+3:away_team.find('PF')-1]
        away_team_pf = away_team[away_team.find('PF')+3:away_team.find(' C ')]
        away_team_c = away_team[away_team.find(' C ')+2:away_team.find('Pro')-1]

        home_team_pg = home_team[home_team.find('PG')+3:home_team.find('SG')-1]
        home_team_sg = home_team[home_team.find('SG')+3:home_team.find('SF')-1]
        home_team_sf = home_team[home_team.find('SF')+3:home_team.find('PF')-1]
        home_team_pf = home_team[home_team.find('PF')+3:home_team.find(' C ')]
        home_team_c = home_team[home_team.find(' C ')+2:home_team.find('Pro')-1]

        away_status = away_team[0:away_team.find('PG')-2]
        home_status = home_team[0:home_team.find('PG')-2]

        # create dataframe
        lineup_df = pd.DataFrame({'Away_Team': away_team_name, 'Home_Team': home_team_name, 'Away_PG': away_team_pg, 'Away_SG': away_team_sg, 'Away_SF': away_team_sf, 'Away_PF': away_team_pf, 'Away_C': away_team_c, 'Home_PG': home_team_pg, 'Home_SG': home_team_sg, 'Home_SF': home_team_sf, 'Home_PF': home_team_pf, 'Home_C': home_team_c, 'Away_Status': away_status, 'Home_Status': home_status}, index=[0])

        tot_lineup_df = pd.concat([tot_lineup_df, lineup_df], axis=0)

    tot_lineup_df = tot_lineup_df.reset_index(drop=True)

    today = datetime.datetime.now().strftime('%Y-%m-%d')

    tot_lineup_df.to_csv('data/team/aggregates/daily_updates/starting_lineups_{}.csv'.format(today), index=False)

    driver.close()

    # add abbrevs to games_soon
    games_soon['team1_abv'] = games_soon['team1'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
    games_soon['team2_abv'] = games_soon['team2'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])

    # Merge, on Away_Team and Home_Team and team1_abv and team2_abv
    games_soon = pd.merge(games_soon, tot_lineup_df, left_on=['team1_abv', 'team2_abv'], right_on=['Away_Team', 'Home_Team'], how='left')
    
    # RUN TOTAL UPDATE
    if games_soon is not None:
        os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\notebooks_v3_1\\4_Running_All')
        %run 1_Daily_Update_Notebook.py
        today = datetime.datetime.today().strftime('%Y-%m-%d')

        predictions = pd.read_csv('data/team/aggregates/daily_updates/predictions_today_'+ str(today) +'.csv')

        # combine games_soon with predictions
        games_soon = pd.merge(games_soon, predictions, left_on=['team1_abv', 'team2_abv'], right_on=['Away_Team', 'Home_Team'], how='left')
        games_soon

        # concat each pair of rows
        # create new df with each row as a pair
        new_df = pd.DataFrame()
        for i in range(0, len(big_odds_df), 2):
            new_df = pd.concat([new_df, pd.concat([big_odds_df.iloc[i], big_odds_df.iloc[i+1]], axis=1).T], axis=0)

        new_df = new_df.reset_index(drop=True)
        new_df
        
    else:
        print('No games starting soon')

else:
    print('No games starting soon')

No games starting soon


In [18]:
big_odds_df

,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2
0,New York,Washington,19:00:00,-4,+4,-4,+4,-150,+125,-173,+153
1,Atlanta,Indiana,19:00:00,-3,+3,-2,+2,-150,+125,-126,+106
2,New Orleans,Detroit,19:00:00,-5,+5,-5.5,+5.5,-233,+165,-212,+182
3,Golden State,San Antonio,19:30:00,-8.5,+8.5,-8.5,+8.5,-400,+260,-365,+295
4,Oklahoma City,Chicago,20:00:00,+6,-6,+4.5,-4.5,+180,-263,+152,-172
5,Phoenix,Minnesota,20:00:00,+2.5,-2.5,+5,-5,+120,-147,+170,-195
6,Orlando,Utah,21:00:00,+5,-5,+6,-6,+165,-233,+195,-225
7,Denver,LA Clippers,22:00:00,-0.5,+0.5,+4.5,-4.5,-111,-110,+162,-182


In [7]:
#api.update_status('This is a test')

NameError: name 'api' is not defined